In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

label_encoder = LabelEncoder()


InputLayer = keras.layers.InputLayer()
LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense

In [6]:
df = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_no_auto_dialer.csv')

df_to_dict = pd.DataFrame()
df= df.dropna(axis=0)
df

,agent_id,queue_id,direction,locality_id,call_type_id,hold_secs,ring_secs,initial_position,trunking_id,carrier_id,week_day,day,month,hour,minute,timestamp,handling_time
0,1000,equipez4,OUT,66000.0,3,0,7,0,1,1.0,4,23,February,20,2,1677185676,78
1,1000,equipez4,AUTO,91.0,5,1,0,1,1,3.0,4,23,February,21,2,1677186767,3
2,1000,equipez4,AUTO,89.0,5,1,0,1,1,67.0,4,23,February,21,2,1677186806,5
3,1000,equipez4,AUTO,48.0,4,14,0,2,1,1.0,4,23,February,21,2,1677186806,2
4,1000,equipez4,AUTO,31.0,5,2,0,1,1,3.0,4,23,February,21,2,1677186824,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117583,1573,isnrecuperacao,OUT,11.0,5,0,15,0,1,1.0,1,24,April,21,4,1682370146,47
117584,1137,isjaguars,OUT,21.0,5,0,15,0,1,3.0,1,24,April,21,4,1682370156,70
117585,1572,isnxsell,OUT,11.0,5,0,40,0,1,3.0,1,24,April,21,4,1682370164,2
117586,1600,mgmsales,OUT,89.0,5,0,16,0,1,5.0,1,24,April,21,4,1682370171,4


In [7]:
df_to_dict['time'] = df['timestamp']
df_to_dict['ht'] = df['handling_time']

In [8]:
dict = df_to_dict.set_index('time').to_dict()['ht']
dict

{1677185676: 78,
 1677186767: 3,
 1677186806: 2,
 1677186824: 23,
 1677243851: 48,
 1677244092: 53,
 1677249481: 597,
 1677250126: 137,
 1677256897: 9,
 1677256909: 399,
 1677256914: 15,
 1677256974: 258,
 1677256993: 99,
 1677257009: 2,
 1677257018: 27,
 1677257027: 75,
 1677257032: 2,
 1677257033: 16,
 1677257038: 26,
 1677257042: 3,
 1677257044: 705,
 1677257050: 16,
 1677257051: 2,
 1677257053: 68,
 1677257057: 71,
 1677257058: 104,
 1677257062: 23,
 1677257063: 451,
 1677257065: 87,
 1677257069: 51,
 1677257073: 23,
 1677257091: 115,
 1677257094: 47,
 1677257107: 84,
 1677257114: 26,
 1677257117: 2,
 1677257124: 21,
 1677257125: 262,
 1677257130: 1018,
 1677257134: 2,
 1677257155: 221,
 1677257156: 9,
 1677257157: 1,
 1677257162: 1783,
 1677257170: 12,
 1677257176: 61,
 1677257187: 55,
 1677257191: 22,
 1677257192: 276,
 1677257201: 37,
 1677257202: 118,
 1677257219: 56,
 1677257235: 6,
 1677257254: 1266,
 1677257258: 2,
 1677257260: 61,
 1677257261: 2,
 1677257263: 528,
 16772572

In [9]:
from datetime import datetime
from time import gmtime, strftime
import time
def transform_dict(data: dict, period: int) -> dict:
    """method that receives dict from database query
    {start_at(timestamp): talk_secs(seconds), ...} and create
    another dict based on the period (15, 30, 1hr) chosen
    {last_timestamp: [12,24,56,12, 90, 100], last_timestamp - period:[195,235,201,300], ... interval of three minutes for standard model}
    For standard model the data from the last period( 15 min, 30 min, 1h) is used to construct forecast for the next period so the return_dict begins with the last timestamp to the first
    """
    seconds = period * 60
    
    return_dict = {}

    keys_array = list(data.keys())
    beginning = keys_array[0]
    limit= beginning + seconds
    end = keys_array[-1]
    arr = []
    break_v = 1
    indice = 0
    while beginning < end:
        if break_v == beginning:
            break
        break_v = beginning
        for key, value in data.items():
            if key <= limit:
                arr.append(value)
                continue
        
            return_dict[f"{indice}"] = arr
            indice += 1
            arr = []
            arr.append(value)
            beginning = key
            limit = beginning + seconds
            

    return return_dict

In [10]:
dict_transformed = transform_dict(dict,120)

In [11]:
dict_transformed

{'0': [78, 3, 2, 23],
 '1': [48, 53, 597, 137],
 '2': [9,
  399,
  15,
  258,
  99,
  2,
  27,
  75,
  2,
  16,
  26,
  3,
  705,
  16,
  2,
  68,
  71,
  104,
  23,
  451,
  87,
  51,
  23,
  115,
  47,
  84,
  26,
  2,
  21,
  262,
  1018,
  2,
  221,
  9,
  1,
  1783,
  12,
  61,
  55,
  22,
  276,
  37,
  118,
  56,
  6,
  1266,
  2,
  61,
  2,
  528,
  26,
  84,
  22,
  54,
  70,
  27,
  53,
  54,
  600,
  199,
  26,
  90,
  70,
  72,
  87,
  64,
  51,
  1,
  75,
  62,
  1,
  219,
  40,
  91,
  7,
  132,
  60,
  54,
  1,
  109,
  30,
  3,
  3,
  18,
  111,
  71,
  727,
  51,
  207,
  913,
  7,
  49,
  958,
  75,
  89,
  52,
  42,
  122,
  17,
  42,
  43,
  40,
  49,
  13,
  2,
  86,
  41,
  68,
  42,
  35,
  37,
  34,
  41,
  46,
  99,
  6,
  632,
  55,
  58,
  52,
  18,
  5,
  3,
  3,
  5,
  44,
  817,
  43,
  17,
  11,
  8,
  25,
  52,
  46,
  1052,
  1231,
  76,
  36,
  8,
  48,
  1,
  1016,
  1,
  3,
  26,
  18,
  3,
  13,
  50,
  55,
  36,
  14,
  16,
  19,
  49,
  11,
  53,


In [12]:
mean_object = {i: np.mean(dict_transformed[i]) for i in dict_transformed.keys()}

In [13]:
mean_object

{'0': 26.5,
 '1': 208.75,
 '2': 149.77319587628867,
 '3': 136.7588028169014,
 '4': 112.9030303030303,
 '5': 150.03116883116883,
 '6': 133.03333333333333,
 '7': 85.65730337078652,
 '8': 117.48826291079813,
 '9': 111.59212376933895,
 '10': 93.11350574712644,
 '11': 121.12116040955631,
 '12': 116.88524590163935,
 '13': 80.32203389830508,
 '14': 112.63616557734204,
 '15': 105.81587301587301,
 '16': 99.40242261103634,
 '17': 119.46037099494097,
 '18': 114.60849056603773,
 '19': 117.47741935483872,
 '20': 113.36907216494845,
 '21': 103.77493606138107,
 '22': 107.11272141706924,
 '23': 125.65783664459161,
 '24': 97.25,
 '25': 62.37391304347826,
 '26': 82.77281947261663,
 '27': 88.53515625,
 '28': 95.75102040816327,
 '29': 115.70254403131115,
 '30': 165.48275862068965,
 '31': 95.10810810810811,
 '32': 82.50735294117646,
 '33': 84.39545454545454,
 '34': 64.4247491638796,
 '35': 92.72881355932203,
 '36': 113.07065217391305,
 '37': 105.59016393442623,
 '38': 96.18518518518519,
 '39': 46.981132075

In [14]:
cv_object = {i: np.std(dict_transformed[i]) for i in dict_transformed.keys()}

In [15]:
cv_object

{'0': 30.890937182286976,
 '1': 226.92771426161238,
 '2': 411.47521890921246,
 '3': 308.1471546787264,
 '4': 276.2735189328585,
 '5': 379.5037348206071,
 '6': 286.6991370499486,
 '7': 182.2626530418445,
 '8': 267.63473830694267,
 '9': 242.13601074949193,
 '10': 221.27282671078999,
 '11': 276.2865419057611,
 '12': 238.09215039013478,
 '13': 165.50185012821836,
 '14': 229.18467898149427,
 '15': 218.9743865621493,
 '16': 229.23302594206677,
 '17': 232.95415423260857,
 '18': 229.13453353043965,
 '19': 265.2792645492517,
 '20': 214.7582718596162,
 '21': 232.58652485841054,
 '22': 237.548259727743,
 '23': 305.9331542098176,
 '24': 230.26851174227014,
 '25': 171.0621275834711,
 '26': 200.59260357270134,
 '27': 215.7008261928732,
 '28': 214.6888063858806,
 '29': 255.72423834989434,
 '30': 370.98460519129594,
 '31': 209.58149423804423,
 '32': 174.96579482201358,
 '33': 194.56452067787583,
 '34': 152.36969932951405,
 '35': 225.3382180677404,
 '36': 281.58034301685143,
 '37': 275.4206545494222,
 

In [16]:
call_volume = {i: len(dict_transformed[i]) for i in dict_transformed.keys()}

In [17]:
call_volume

{'0': 4,
 '1': 4,
 '2': 582,
 '3': 568,
 '4': 165,
 '5': 385,
 '6': 540,
 '7': 178,
 '8': 213,
 '9': 711,
 '10': 696,
 '11': 586,
 '12': 61,
 '13': 118,
 '14': 459,
 '15': 315,
 '16': 743,
 '17': 593,
 '18': 212,
 '19': 155,
 '20': 485,
 '21': 391,
 '22': 621,
 '23': 453,
 '24': 20,
 '25': 115,
 '26': 493,
 '27': 512,
 '28': 735,
 '29': 511,
 '30': 116,
 '31': 111,
 '32': 136,
 '33': 440,
 '34': 598,
 '35': 531,
 '36': 184,
 '37': 61,
 '38': 81,
 '39': 53,
 '40': 134,
 '41': 180,
 '42': 311,
 '43': 294,
 '44': 27,
 '45': 51,
 '46': 306,
 '47': 202,
 '48': 433,
 '49': 500,
 '50': 152,
 '51': 120,
 '52': 338,
 '53': 233,
 '54': 407,
 '55': 444,
 '56': 130,
 '57': 50,
 '58': 287,
 '59': 226,
 '60': 489,
 '61': 521,
 '62': 205,
 '63': 1,
 '64': 53,
 '65': 242,
 '66': 166,
 '67': 503,
 '68': 637,
 '69': 165,
 '70': 312,
 '71': 511,
 '72': 74,
 '73': 90,
 '74': 56,
 '75': 1,
 '76': 26,
 '77': 19,
 '78': 133,
 '79': 91,
 '80': 69,
 '81': 72,
 '82': 262,
 '83': 229,
 '84': 385,
 '85': 390,
 '8

In [18]:
df_to_ml = pd.DataFrame()
df_to_ml['call_volume'] = call_volume.values()
df_to_ml['cv'] = cv_object.values()
df_to_ml['mean'] = mean_object.values()
df_to_ml

,call_volume,cv,mean
0,4,30.890937,26.500000
1,4,226.927714,208.750000
2,582,411.475219,149.773196
3,568,308.147155,136.758803
4,165,276.273519,112.903030
...,...,...,...
321,7,282.985974,182.285714
322,65,252.788321,93.415385
323,178,199.305185,85.674157
324,183,292.603345,135.120219


In [19]:
df_to_ml.iloc[:, 0:3].values

array([[4.00000000e+00, 3.08909372e+01, 2.65000000e+01],
       [4.00000000e+00, 2.26927714e+02, 2.08750000e+02],
       [5.82000000e+02, 4.11475219e+02, 1.49773196e+02],
       [5.68000000e+02, 3.08147155e+02, 1.36758803e+02],
       [1.65000000e+02, 2.76273519e+02, 1.12903030e+02],
       [3.85000000e+02, 3.79503735e+02, 1.50031169e+02],
       [5.40000000e+02, 2.86699137e+02, 1.33033333e+02],
       [1.78000000e+02, 1.82262653e+02, 8.56573034e+01],
       [2.13000000e+02, 2.67634738e+02, 1.17488263e+02],
       [7.11000000e+02, 2.42136011e+02, 1.11592124e+02],
       [6.96000000e+02, 2.21272827e+02, 9.31135057e+01],
       [5.86000000e+02, 2.76286542e+02, 1.21121160e+02],
       [6.10000000e+01, 2.38092150e+02, 1.16885246e+02],
       [1.18000000e+02, 1.65501850e+02, 8.03220339e+01],
       [4.59000000e+02, 2.29184679e+02, 1.12636166e+02],
       [3.15000000e+02, 2.18974387e+02, 1.05815873e+02],
       [7.43000000e+02, 2.29233026e+02, 9.94024226e+01],
       [5.93000000e+02, 2.32954

In [21]:
normalizador = MinMaxScaler(feature_range = (0,1))
base_treinamento_normalizada = normalizador.fit_transform(df_to_ml)

In [22]:
base_treinamento_normalizada.shape

(326, 3)

In [23]:
previsores = []
tempo_real = []


In [24]:
for i in range(1, 326):
    previsores.append(base_treinamento_normalizada[i-1:i, 0:3])
    tempo_real.append(base_treinamento_normalizada[i,2])

In [25]:
previsores

[array([[0.00247117, 0.04132128, 0.04709141]]),
 array([[0.00247117, 0.30354998, 0.38365651]]),
 array([[0.4785832 , 0.55041005, 0.27474274]]),
 array([[0.46705107, 0.41219321, 0.25070878]]),
 array([[0.13509061, 0.36955742, 0.2066538 ]]),
 array([[0.31630972, 0.50764337, 0.27521915]]),
 array([[0.44398682, 0.38350325, 0.24382887]]),
 array([[0.14579901, 0.24380373, 0.15633851]]),
 array([[0.17462932, 0.35800175, 0.21512145]]),
 array([[0.58484349, 0.32389336, 0.20423292]]),
 array([[0.57248764, 0.29598571, 0.17010804]]),
 array([[0.48187809, 0.36957484, 0.2218304 ]]),
 array([[0.04942339, 0.31848409, 0.21400784]]),
 array([[0.09637562, 0.22138364, 0.14648575]]),
 array([[0.37726524, 0.30656901, 0.20616097]]),
 array([[0.25864909, 0.2929112 , 0.19356579]]),
 array([[0.61120264, 0.30663368, 0.18172193]]),
 array([[0.48764415, 0.31161125, 0.21876338]]),
 array([[0.1738056 , 0.30650193, 0.20980331]]),
 array([[0.12685338, 0.35485095, 0.21510142]]),
 array([[0.39868204, 0.28727152, 0.20751

In [26]:
previsores, tempo_real = np.array(previsores), np.array(tempo_real)
previsores= np.reshape(previsores, (previsores.shape[0], previsores.shape[1],3))

In [27]:
previsores.shape[0:]

(325, 1, 3)

In [28]:
tempo_real.shape[0]

325

In [29]:
regressor = keras.models.Sequential()
#camadas inicial
regressor.add(LSTM(units = 100, return_sequences = True, input_shape=(1,3)))
regressor.add(Dropout(0.3))     

#camadas
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))    

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))  

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3)) 

#camadas finais
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3)) 

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])



In [30]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [31]:
es = EarlyStopping(monitor = 'loss', min_delta = 1e-10, patience = 10 , verbose =1)
rlr = ReduceLROnPlateau(monitor = 'loss', factoe = 0.2, patience = 5, verbose = 1)
mcp = ModelCheckpoint(filepath = 'pesos.h5', monitor = 'loss', save_best_only = True)

In [32]:
regressor.fit(previsores, tempo_real, epochs = 100, batch_size = 3)

Epoch 1/100


2023-05-03 10:39:53.420558: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


109/109 [==============================] - 3s 2ms/step - loss: 0.0453 - mean_absolute_error: 0.1702
Epoch 2/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0087 - mean_absolute_error: 0.0635
Epoch 3/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0087 - mean_absolute_error: 0.0637
Epoch 4/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0081 - mean_absolute_error: 0.0595
Epoch 5/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0086 - mean_absolute_error: 0.0611
Epoch 6/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0080 - mean_absolute_error: 0.0594
Epoch 7/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0085 - mean_absolute_error: 0.0619
Epoch 8/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0086 - mean_absolute_error: 0.0608
Epoch 9/100
109/109 [==============================] - 0s 2ms/step - loss: 0.0078 - mean_absolute_error: 0.0578
Epoc